In [1]:
print("Hello")

Hello


#### Load Envirenment

In [58]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ['GROQ_API_KEY']
hf_token_api = os.environ['HF_TOKEN']
pinecone_api_key = os.environ['PINECONE_API_KEY']

#### Load Model Open Source  

In [ ]:
from langchain_groq import ChatGroq, NomicCroqEmbeddings
llamaChatModel = ChatGroq(
    model="llama3-70b-8192"
)

#### Get Data (Txt exemple)

In [73]:
# Txt data loading
from langchain_community.document_loaders import TextLoader

text_path = r'Data\be-good.txt'
loader = TextLoader(text_path)

loader_data = loader.load()

In [7]:
loader_data, type(loader_data)

([Document(metadata={'source': 'Data\\be-good.txt'}, page_content='Be good\n\nApril 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We\'ve\nlearned a lot since then, but if I were choosing now that\'s still\nthe one I\'d pick.Another thing we tell founders is not to worry too much about the\nbusiness model, at least at first.  Not because making money is\nunimportant, but because it\'s so much easier than building something\ngreat.A couple weeks ago I realized that if you put those two ideas\ntogether, you get something surprising.  Make something people want.\nDon\'t worry too much about making money.  What you\'ve got is a\ndescription of a charity.When you get an unexpected result like this, it could either be a\nbug or a new discovery.  Either businesses aren\'t supposed to be\nlike charities, and we\'ve proven by reductio ad absurdum that o

In [8]:
loader_data[0].page_content

'Be good\n\nApril 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We\'ve\nlearned a lot since then, but if I were choosing now that\'s still\nthe one I\'d pick.Another thing we tell founders is not to worry too much about the\nbusiness model, at least at first.  Not because making money is\nunimportant, but because it\'s so much easier than building something\ngreat.A couple weeks ago I realized that if you put those two ideas\ntogether, you get something surprising.  Make something people want.\nDon\'t worry too much about making money.  What you\'ve got is a\ndescription of a charity.When you get an unexpected result like this, it could either be a\nbug or a new discovery.  Either businesses aren\'t supposed to be\nlike charities, and we\'ve proven by reductio ad absurdum that one\nor both of the principles we began with is false.  Or we have 

#### Text CharacterTextSplitter

In [11]:
from langchain_text_splitters import CharacterTextSplitter

text_splitters = CharacterTextSplitter(
    separator="\n\n",
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len,
    is_separator_regex=False,
)

#### pass inside the create documents , the text only

In [17]:
texts = text_splitters.create_documents([loader_data[0].page_content])
texts

[Document(page_content='Be good'),
 Document(page_content='April 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We\'ve\nlearned a lot since then, but if I were choosing now that\'s still\nthe one I\'d pick.Another thing we tell founders is not to worry too much about the\nbusiness model, at least at first.  Not because making money is\nunimportant, but because it\'s so much easier than building something\ngreat.A couple weeks ago I realized that if you put those two ideas\ntogether, you get something surprising.  Make something people want.\nDon\'t worry too much about making money.  What you\'ve got is a\ndescription of a charity.When you get an unexpected result like this, it could either be a\nbug or a new discovery.  Either businesses aren\'t supposed to be\nlike charities, and we\'ve proven by reductio ad absurdum that one\nor both of the 

In [14]:
len(texts)

2

### RecursiveCharacterTextSplitter

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 26,
    chunk_overlap = 4
)

texts = recursive_splitter.split_text(loader_data[0].page_content)

In [19]:
texts

['Be good',
 'April 2008(This essay is',
 'is derived from a talk at',
 'at the 2008 Startup',
 'School.)About a month',
 'after we started Y',
 'Y Combinator we came up',
 'up with the',
 'phrase that became our',
 'our motto: Make something',
 "people want.  We've",
 'learned a lot since then,',
 'but if I were choosing',
 "now that's still",
 "the one I'd pick.Another",
 'thing we tell founders is',
 'is not to worry too much',
 'about the',
 'business model, at least',
 'at first.  Not because',
 'making money is',
 'unimportant, but because',
 "it's so much easier than",
 'building something',
 'great.A couple weeks ago',
 'ago I realized that if',
 'if you put those two',
 'two ideas',
 'together, you get',
 'get something surprising.',
 'Make something people',
 'want.',
 "Don't worry too much",
 'about making money.  What',
 "you've got is a",
 'description of a',
 'a charity.When you get an',
 'an unexpected result like',
 'this, it could either be',
 'be a',
 'bug or a new di

## Embeddings
#### Transform the small parts of text in number (vector representation)
    - can user OpenAI embed , or any free open source embedding model
#### https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/

In [47]:
from langchain_huggingface import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [48]:
chunk_of_text = [
    'April 2008(This essay is',
    'is derived from a talk at',
    'at the 2008 Startup',
    'School.)About a month',
    'after we started Y',
    'Y Combinator we came up',
]

In [49]:
simple_test = model_embeddings.embed_documents(chunk_of_text)

print(simple_test, len(simple_test))

[[-0.04102138429880142, 0.0797424390912056, 0.05828357860445976, 0.09284177422523499, 0.0241863913834095, 0.08879270404577255, 0.0027790553867816925, -0.02149549312889576, -0.07406344264745712, 0.04199939966201782, -0.009638306684792042, 0.0852317065000534, -0.10068673640489578, 0.06754357367753983, 0.010283483192324638, 0.05791640654206276, 0.028718600049614906, -0.04877253994345665, -0.07197970151901245, 0.024166109040379524, 0.017414961010217667, -0.01237185113132, -0.007017659489065409, 0.04738541319966316, -0.04118596389889717, 0.05321988835930824, 0.054211799055337906, -0.05348832532763481, -0.032334282994270325, -0.0016132047167047858, -0.06141092628240585, -0.01276187039911747, 0.0307391956448555, 0.03578714281320572, 0.01212122943252325, 0.06368103623390198, 0.09631498157978058, 0.04690994694828987, -0.011017288081347942, -0.004533803090453148, -0.015502279624342918, -0.15500706434249878, 0.01341159362345934, 0.02367589622735977, 0.021229667589068413, -0.01939491368830204, 0.0

In [50]:
len(simple_test[0])

384

#### Vector Stores (Vector DataBase)

In [60]:
from pinecone import Pinecone, ServerlessSpec

In [63]:
pc = Pinecone(api_key=pinecone_api_key)
index_name = "rag"

In [64]:
pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [ ]:
# ChromaDB
from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
# from langchain_chroma import Chroma ## not suported try with another kernel
from langchain_pinecone import PineconeVectorStore

# Load the document , split it into chunks , preforme embed , load into the vector store
doc_path = r'Data\be-good-and-how-not-to-die.txt'

load_document = TextLoader(doc_path).load()

text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)

chunk_of_text = text_splitter.split_documents(load_document)

vector_store = PineconeVectorStore.from_documents(documents=chunk_of_text, index_name = index_name, embedding=model_embeddings)


Created a chunk of size 16701, which is longer than the specified 1000


#### Query the model Base on the new knoleg

In [78]:
query = 'Why does Paul Graham believe that being good is an effective strategy for startups'
response = vector_store.similarity_search(query)
print(response[0].page_content)

If so many startups get demoralized and fail when merely by hanging on they could get rich, you have to assume that running a startup can be demoralizing. That is certainly true. I've been there, and that's why I've never done another startup. The low points in a startup are just unbelievably low. I bet even Google had moments where things seemed hopeless.

Knowing that should help. If you know it's going to feel terrible sometimes, then when it feels terrible you won't think "ouch, this feels terrible, I give up." It feels that way for everyone. And if you just hang on, things will probably get better. The metaphor people use to describe the way a startup feels is at least a roller coaster and not drowning. You don't just sink and sink; there are ups after the downs.


In [80]:
from langchain_community.vectorstores import FAISS
vector_db = FAISS.from_documents(documents=chunk_of_text, embedding=model_embeddings)

In [81]:
vector_db

In [82]:
retriver = vector_db.as_retriever()
response = retriver.invoke(query)

In [89]:
print(response[0].page_content)

If so many startups get demoralized and fail when merely by hanging on they could get rich, you have to assume that running a startup can be demoralizing. That is certainly true. I've been there, and that's why I've never done another startup. The low points in a startup are just unbelievably low. I bet even Google had moments where things seemed hopeless.

Knowing that should help. If you know it's going to feel terrible sometimes, then when it feels terrible you won't think "ouch, this feels terrible, I give up." It feels that way for everyone. And if you just hang on, things will probably get better. The metaphor people use to describe the way a startup feels is at least a roller coaster and not drowning. You don't just sink and sink; there are ups after the downs.


In [90]:
pc.delete_index(index_name)